In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

import rpy2.robjects as robjects
from rpy2.robjects.packages import importr

from rpy2.robjects import pandas2ri
pandas2ri.activate()
import rpy2.robjects.numpy2ri as rpyn 
# import rpy2.robjects.numpy2ri as rpyn
rpyn.activate()

R_PMA = importr('PMA')
R_CCA = robjects.r('CCA')

In [ ]:
import os
os.getcwd()

In [ ]:
import sys
sys.path.append('../')

In [ ]:
from cross_validate import CrossValidateExpressionCca

In [ ]:
R_PMA = importr('PMA')
permute = robjects.r("CCA.permute")

In [ ]:
x = pd.read_csv('../../data/m_nmm_summed_on_gene/m.tsv')
z = pd.read_csv('../../data/m_nmm_summed_on_gene/nmm.tsv')

for df in [x, z]:
        df.rename(columns={'Unnamed: 0':'sample'}, inplace=True)
        df.set_index('sample', inplace=True)
        
xr, xc = x.shape
zr, zc = z.shape
x_R = robjects.r.matrix(x.as_matrix(), nrow=xr, ncol=xc)
z_R = robjects.r.matrix(z.as_matrix(), nrow=zr, ncol=zc)

In [ ]:
x.shape, x.as_matrix().shape

In [ ]:
import itertools
list(itertools.product([1, 2, 3], [4, 5, 6]))

In [ ]:
list(itertools.product([0.2, 0.3], [0.2, 0.3]))

In [ ]:
cv = CrossValidateExpressionCca(x=x, z=z, folds=4)

In [ ]:
cv.models

In [ ]:
cv.model_each_fold(0, 0)

In [ ]:
cv.plot_cross_val()

In [ ]:
vl = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
[(v, v) for v in vl]

In [ ]:
cv.model_list_of_penalty_tuples([(v, v) for v in vl])

In [ ]:
cv.plot_cross_val()

In [ ]:
    def plot_cross_val(self):
        colors = {1:'#9e9ac8', 2:'#807dba', 3:'#6a51a3', 4:'#4a1486'}
        fig, axs = plt.subplots(2, 2, figsize=(6,5))
        plts = {'penalty_x, train': axs[0,0],
                'penalty_z, train': axs[1,0],
                'penalty_x, val': axs[0,1],
                'penalty_z, val': axs[1,1]}
        titles = {'penalty_x, train': 'train corr, penalty_x',
                'penalty_z, train': 'train corr, penalty_z',
                'penalty_x, val': 'val corr, penalty_x',
                'penalty_z, val': 'val corr, penalty_z'}

        #import pdb; pdb.set_trace()
        for p_key in plts.keys():
            print(p_key)
            ax = plts[p_key]

            # get x variable
            if 'x' in p_key:
                x = 'penalty_x'
            else:
                x = 'penalty_z'

            # get y variable
            if 'train' in p_key:
                y = 'train correlation'
            else:
                y = 'validation correlation'

            # plot each fold's trend.
            for fold, df in self.summary.groupby('fold'):
                df.sort_values(by=x, inplace=True)
                # plot
                ax.plot(df[x], df[y], color=colors[fold],
                        marker='o', label='fold {}'.format(fold))

            ax.set_title(titles[p_key])
            
        # couldn't get a legend to work.
        plt.tight_layout()
        return fig
            
plot_cross_val(cv)